In [1]:
import pandas as pd
from pathlib import Path
import os
import re
import numpy as np
from funcz import clean_column_names
import duckdb
import warnings
warnings.filterwarnings('ignore')
engine = duckdb.connect('my_db.duckdb')

# 1.1: Data Prep: Demographic Data with the expontential model


In [2]:


cb = pd.read_excel(os.path.join(os.getcwd(), 'demographic_data', 'demographic.xlsx'))
cb = cb[cb["CD Type"] == "CD"]

df_dict = {"year":[], "community_board":[], "metric":[],"value":[]}
metric = pd.read_csv(r'C:\Users\joe tech\Desktop\NYC\demographic_data\column_lookup.csv')
metric_list = list(metric["abbrev"].unique())
cb = cb[cb["CD Type"] == "CD"]

cb = cb.drop(columns=["Orig Order", "GeoType", "Borough","GeoID","BCT2020","CD Type","NTA Type"])

columns = cb.columns.to_list()
for n in columns:
    if n[-1] == "P":
        cb = cb.drop(columns=[n])
cb.columns
columns = cb.columns.to_list()
cb.reset_index(inplace=True)
for i ,row in cb.iterrows(): 
    for n in columns: 
            try:
                
                if n[:-3] in metric_list and n[-3:] in ("_10", "_20"):

                    df_dict["metric"].append(n[:-3])
                    df_dict["value"].append(cb[n].iloc[i])
                    df_dict["community_board"].append(cb["Name"].iloc[i])
                    if n[-3:] == "_10":
                        df_dict["year"].append(2010)
                    elif n[-3:] == "_20":
                        df_dict["year"].append(2020)
            except Exception as e:
                print(f"error {e}")

demo_df = pd.DataFrame(df_dict)

demo_df["metric"] = demo_df["metric"].map({
    "Pop": "Population",
    "HHPop": "Household Population",
    "AvHHSz": "Average Household Size",
    "PopU18": "Under 18 Population",
    "Hsp": "Hispanic Population",
    "WNH": "White Population",
    "BNH": "Black Population",
    "ANH": "Asian Population",
    "ONH": "Other Population",
    "HUnits": "Housing Units",
    "OcHU": "Occupied Housing"
})

demo_df
predicted_rows = []


grouped = demo_df.groupby(['community_board', 'metric'])

for (cb, m), group in grouped:
    if not set([2010, 2020]).issubset(group['year'].values):
        continue

    value_2010 = group.loc[group['year'] == 2010, 'value'].values[0]
    value_2020 = group.loc[group['year'] == 2020, 'value'].values[0]

    if value_2010 == 0 or value_2020 == 0:
        continue

    r = (1 / 10) * np.log(value_2020 / value_2010)

    for year in range(2006, 2025):
        if year in (2010, 2020):
            continue

        t = year - 2010
        predicted_value = value_2010 * np.exp(r * t)

        predicted_rows.append({
            'year': year,
            'community_board': cb,
            'metric': m,
            'value': predicted_value
        })


predicted_df = pd.DataFrame(predicted_rows)


demo_df = pd.concat([demo_df, predicted_df], ignore_index=True)
demo_df.sort_values(by=['community_board', 'metric', 'year'], inplace=True)
 
demo_df = clean_column_names(demo_df)
demo_df["community_board"] = demo_df["community_board"].str.replace("District",repl="Board")
demo_df.to_sql(con=engine,name="demo_data",if_exists="replace",index=False)


-1

# 1.2.1 : Data Prep School Data: Pre 2014


In [3]:
school_data_folder = Path.cwd() / 'school_data'
school_performance_df = pd.DataFrame()

years = ["2006-2007","2007-2008","2008-2009","2009-2010","2010-2011","2011-2012","2012-2013"]
counter = 0

for file in school_data_folder.iterdir():
    if "pr" in file.name:
        name = file.name.split("_")
        year = f"20{name[1]}-20{name[2].replace('.csv', '')}"
        year_alt = f"20{name[1]}-{name[2].replace('.csv', '')}"
        #print(f"start {year_alt}")
        
        df = pd.read_csv(file)
        df = df[df["SCHOOL LEVEL*"].isin(["Elementary", "Middle School", "High School"])]
       
        df = df.reset_index(drop=True)
        
        map_dictionary = {}
        columns = df.columns.to_list()
        
        for n in columns:
            year = f"20{name[1]}-20{name[2].replace('.csv', '')}"
            year_alt = f"20{name[1]}-{name[2].replace('.csv', '')}"

            if (year in n):
                print(n)
                map_dictionary[n] = n.replace(year, " ").strip(" ")
            elif year_alt in n:
                print(n)
                map_dictionary[n] = n.replace(year_alt, " ").strip(" ")
            elif re.search(r"20\d{2}-\d{2,4}", n):
                df.drop(columns=[n], inplace=True)

        df.rename(columns=map_dictionary, inplace=True)
        df = df.reset_index()
        df["year"] = years[counter]
        school_performance_df = pd.concat([school_performance_df, df], ignore_index=True)
        counter += 1
school_performance_df

school_performance_df = school_performance_df[["DBN", "OVERALL SCORE","year"]]
school_performance_df.rename(columns={"OVERALL SCORE":"score"},inplace=True)
school_performance_df.rename(columns={'year':'years'},inplace=True)
school_performance_df = clean_column_names(school_performance_df)
school_performance_df["score"].replace(to_replace=".", value=np.nan,inplace=True)
school_performance_df["score"] = school_performance_df["score"].astype(float)
school_performance_df["score"] = school_performance_df["score"]/100


2007-08 SCHOOL SUPPORT ORGANIZATION
2007-08 QUALITY REVIEW SCORE
2008-09 SCHOOL SUPPORT ORGANIZATION
2008-09 STATE ACCOUNTABILITY STATUS
2009-2010 OVERALL GRADE
2009-2010 OVERALL SCORE
2009-2010 ENVIRONMENT CATEGORY SCORE
2009-2010 ENVIRONMENT GRADE
2009-2010 PERFORMANCE CATEGORY SCORE
2009-2010 PERFORMANCE GRADE
2009-2010 PROGRESS CATEGORY SCORE
2009-2010 PROGRESS GRADE
2009-2010 ADDITIONAL CREDIT
2010-2011 OVERALL GRADE
2010-2011 OVERALL SCORE
2010-2011 ENVIRONMENT CATEGORY SCORE
2010-2011 ENVIRONMENT GRADE
2010-2011 PERFORMANCE CATEGORY SCORE
2010-2011 PERFORMANCE GRADE
2010-2011 PROGRESS CATEGORY SCORE
2010-2011 PROGRESS GRADE
2010-2011 ADDITIONAL CREDIT
2011-2012 OVERALL GRADE
2011-2012 OVERALL SCORE
2011-12 OVERALL PERCENTILE
2011-2012 PROGRESS CATEGORY SCORE
2011-2012 PROGRESS GRADE
2011-2012 PERFORMANCE CATEGORY SCORE
2011-2012 PERFORMANCE GRADE
2011-2012 ENVIRONMENT CATEGORY SCORE
2011-2012 ENVIRONMENT GRADE
2011-2012 COLLEGE AND CAREER READINESS SCORE
2011-2012 COLLEGE AND CA

# 1.2.2: Data Prep School Data: Post 2014

In [5]:
data = pd.DataFrame(data={'DBN':[],'score':[],'year':[]})

years = ["2014-2015", "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2022-2023", "2023-2024"]
folder = Path.cwd() / 'new_school_data'

files = [f for f in folder.iterdir() if f.is_file()]

for i, file in enumerate(files):
    year = years[i // 2]  
    try:
        df = pd.read_excel(file, sheet_name="Student Achievement", header=0)  
        df = df[["DBN", "Student Achievement - Section Score"]]
    except Exception as e:
        try:
            df = pd.read_excel(file, sheet_name="Student Achievement", header=1)
            df = df[["DBN", "Student Achievement - Section Score"]]
        except Exception as e:
            print(f"error in file {file}: {e}")
       
    df.rename(columns={"Student Achievement - Section Score": "score"},inplace=True)
    df["year"] = year

    data = pd.concat([data,df], ignore_index=True)
data.rename(columns={"year":"years"},inplace=True)
data = clean_column_names(data)
data["score"].replace(to_replace=".", value=np.nan,inplace=True)
data["score"] = data["score"].astype(float)
data["score"] = data["score"]/4.99
school_performance_df = pd.concat([school_performance_df, data],ignore_index=True)

school_performance_df = clean_column_names(school_performance_df)



In [6]:
all_dbns = school_performance_df["dbn"].dropna().unique()
missing_years = ["2019-2020", "2020-2021", "2021-2022"]
missing_df = pd.DataFrame([(dbn, np.nan, year) for dbn in all_dbns for year in missing_years],
                          columns=["dbn", "score", "years"])
school_performance_df = pd.concat([school_performance_df, missing_df], ignore_index=True)
school_performance_df["year_order"] = school_performance_df["years"].str.split("-").str[0].astype(int)
school_performance_df.sort_values(by=["dbn", "year_order"], inplace=True)
school_performance_df["interpolated"] = school_performance_df.groupby("dbn")["score"].transform(lambda x: x.interpolate())
mask = school_performance_df["year"].isin(missing_years) & school_performance_df["score"].isna()
school_performance_df.loc[mask, "score"] = school_performance_df.loc[mask, "interpolated"]
school_performance_df.drop(columns=["interpolated", "year_order"], inplace=True)
school_performance_df.to_sql(name="school_performance",con=engine,if_exists="replace",index=False)

-1

# 1.3.1 Data Prep: Keeping Track Online Data
As they are all aggregated the same way from the same source, minimal data prep is needed. 







In [7]:
community_board_mapping = pd.read_csv(Path.cwd() / 'keep_track_online' / f'community_boards.csv')
names = ["median_income","rent","unemployment_rate"]
for n in names:
    df = pd.read_csv(Path.cwd() / 'keep_track_online' / f'{n}.csv')
    df["Data"] = pd.to_numeric(df["Data"], errors="coerce")
    df.rename(columns={"Data":n,"TimeFrame": "year"},inplace=True)
    df = df.merge(right=community_board_mapping,left_on=["Location"], right_on=["Community Name"],how="left")
    df.drop(columns=["DataFormat","Fips","Location","Community Name"],inplace=True)
    df.rename(columns={"Data": n,"TimeFrame": "year"},inplace=True)
    df = clean_column_names(df)
    df.to_sql(con=engine,name=f"{n}_data",if_exists="replace",index=False)
    

## 1.3.2 Filling it Missing Values for Keeping Track Online Data

In [21]:
query =   '''
  -------------------------------------------------------------------------------------------------------------------
  --Base CTE
  -------------------------------------------------------------------------------------------------------------------
  WITH base_CTE AS (
  SELECT 
  cb.community_board,
  ye.year
  FROM community_board_data cb
  CROSS JOIN generate_series(2006, 2024) AS ye(year)
  )
  SELECT DISTINCT b.*, urd.unemployment_rate, rd.rent, mid.median_income FROM base_CTE b
  LEFT JOIN median_income_data mid ON b.community_board = mid.community_board AND b.YEAR = mid.YEAR
  LEFT JOIN unemployment_rate_data urd  ON b.community_board = urd.community_board  AND b.YEAR = urd.YEAR
  LEFT JOIN rent_data rd ON b.community_board = rd.community_board  AND b.YEAR = rd.YEAR
  '''

try:
   df = pd.read_sql(con=engine,sql= query)

except:
    print("yuh")
    cb_df = pd.read_csv(Path.cwd() / 'geographic_district_data' / f'community_boards.csv')
    cb_df = cb_df[["Community Board", "Borough", "Latitude", "Longitude"]]
    cb_df["Community Board"] = cb_df["Borough"] + " " + cb_df["Community Board"]
    cb_df = clean_column_names(cb_df)
    cb_df.to_sql(name="community_board_data", con=engine, if_exists="replace",index=False)
    metrics = ["median_income", "rent", "unemployment_rate"]
    df = pd.read_sql(con=engine,sql=query)

for metric in metrics:
    df[metric] = (
        df.groupby('community_board')[metric]
          .transform(lambda g: g.interpolate(method='linear', limit_direction='both'))
    )

    df[metric] = (
        df.groupby('community_board')[metric]
          .transform(lambda g: g.ffill().bfill())
    )

    year_medians = df.groupby('year')[metric].transform('median')
    df[metric] = df[metric].fillna(year_medians)

    upload_df = df[['community_board', 'year', metric]].drop_duplicates().copy()
    upload_df.to_sql(con=engine, name=f"{metric}_data", if_exists="replace", index=False)

# 1.4 Crime Data Prep

In [29]:
crime_types = ['major','non-major','misdemeanor','violation']
crime_df = None
for n in crime_types:
    df = pd.read_excel(Path.cwd() / 'crime_data' / f'{n}.xls')
    
    filled = df["PCT"][~df["PCT"].isna()]
    filled.to_dict()

    for key, val in filled.items():
        df["PCT"].iloc[key: key+8] = val

    df_melted = pd.melt(df, id_vars=['PCT', 'CRIME'], var_name='Year', value_name='Count')
    df_melted["crime_category"] = n
    if isinstance(crime_df, pd.DataFrame):
        
        crime_df = pd.concat([crime_df,df_melted],ignore_index=True)
    else:
        crime_df = df_melted
crime_df.astype({'PCT': 'object'})
crime_df = clean_column_names(crime_df)
crime_df = crime_df[~crime_df["crime"].str.contains("TOTAL", case=False, na=False)]
crime_df["pct_2"] = crime_df["pct"].apply(
    lambda x: x.split('.')[0] if isinstance(x, str) and len(x.split('.')[0]) <= 4 else None
)
crime_df['pct'] = pd.to_numeric(crime_df['pct'], errors='coerce').astype('Int64')
crime_df = crime_df.dropna(subset=['pct'])
crime_df.to_sql(con=engine,name="crime_data",if_exists="replace",index=False)



ConnectionException: Connection Error: Connection already closed!

# Uploading Dimension Tables


In [28]:
#CommunityBoard & Borough Table##

cb_df = pd.read_csv(Path.cwd() / 'geographic_district_data' / f'community_boards.csv')
cb_df = cb_df[["Community Board", "Borough", "Latitude", "Longitude"]]
cb_df["Community Board"] = cb_df["Borough"] + " " + cb_df["Community Board"]
cb_df = clean_column_names(cb_df)
cb_df.to_sql(name="community_board_data", con=engine, if_exists="replace",index=False)

borough_df = pd.DataFrame({"borough": list(cb_df["borough"].unique())})
borough_df.to_sql(name="borough_table", con=engine, if_exists="replace",index=False)

##Precinct Table##
precinct_df = pd.read_csv(Path.cwd() / 'geographic_district_data' / f'community_boards.csv')
precinct_df = precinct_df[["Community Board", "CB Precinct(s)","Borough"]]

precinct_df.rename(columns={"CB Precinct(s)": "Precinct"}, inplace=True)
precinct_df["Precinct"] = precinct_df["Precinct"].astype(str)
precinct_df["Precinct"] = precinct_df["Precinct"].str.split(",")
precinct_df = precinct_df.explode("Precinct")
precinct_df["Precinct"] = precinct_df["Precinct"].str.split(";")
precinct_df = precinct_df.explode("Precinct")
precinct_df["Community Board"] = precinct_df["Borough"] + " " + precinct_df["Community Board"]

precinct_df["Precinct"] = precinct_df["Precinct"].str.strip()
precinct_df = clean_column_names(precinct_df)
precinct_df['precinct'] = pd.to_numeric(precinct_df['precinct'], errors='coerce').astype('Int64')
precinct_df.drop(columns=["borough"],inplace=True)
precinct_df.to_sql(name="precinct_data", con=engine, if_exists="replace",index=False)

#School Table#
school_df = pd.read_csv(Path.cwd() / 'school_data' / f'school_district.csv')
school_df['borough'] = school_df['dbn'].str[2].map({
    'M': 'Manhattan',
    'K': 'Brooklyn',
    'Q': 'Queens',
    'X': 'Bronx',
    'R': 'Staten Island'
})
school_df['Community Board'] = school_df['borough'] + ' Community Board ' + school_df['Community Board'].astype(str)
school_df = clean_column_names(school_df)
school_df.drop(columns=['borough'], inplace=True)
school_df.to_sql(name="school_data", con=engine, if_exists="replace",index=False)

engine.close()

